In [75]:
from helper import export_projects, dataset_metadata, _export, download_datasets, upload_dataset, _export, _convert_dictkeys_to_snake
import datarobot as dr
import shutil
import pandas as pd
import json

In [58]:
#Need to manually create this; because our codespace will NOT create an empty directory during init.
#Try to create a one-word .gitkeep and see if we can create this dir for them.
DIR = "migrations"


## 既存環境のプロジェクト・データセットをエキスポート

In [86]:
#MissingSchemaエラーがあったら環境変数が未設定の可能性あり
projects = export_projects()
catalog_metadata = dataset_metadata(projects)
catalog_metadata.to_csv(f"{DIR}/original-datasets.csv", index=False)

In [87]:
#We provide the user a way to filter out projects (don't even put them in the original-projects.json record file)
#Include a list of project ids (> exclude)
#pd.DataFrame(projects)[["id","projectName"]]
#projects_included_in_migration = [""]

list_of_project_ids = ["66974476de6c40f15f3ced74","668f661cb95d4d5a335e5602", "668f5ff2620117f44b5e5ba0"]



In [88]:
with open(f"{DIR}/original-projects.json", "w") as f:
    json.dump(projects, f, ensure_ascii=False, indent=4)

In [89]:
catalog_metadata

,catalogId,name,datasetSize(MB),rowCount,processingState,categories
0,66975d9dbd0e747c51ed6776,週次売上予測データセット_ヒストリカル.xlsx,2,10703,COMPLETED,"[PREDICTION, TRAINING]"
1,668f5f802f5c40837c6c596d,skin_lesion.csv,72,10015,COMPLETED,"[TRAINING, PREDICTION]"
2,6690f5755017d14f3d7c43bb,cal_housing.csv,1,20640,COMPLETED,"[PREDICTION, TRAINING]"
3,668f65a8056f56fd209d4f9f,与信リスクのデータ_train.csv,9,50000,COMPLETED,"[PREDICTION, TRAINING]"
4,668b2f30077b63d05210a702,商品の売上_学習.csv,36,340673,COMPLETED,"[TRAINING, PREDICTION]"
5,668f5f802f5c40837c6c596d,skin_lesion.csv,72,10015,COMPLETED,"[TRAINING, PREDICTION]"
6,668b2be0becebc304710a904,MarketingTrain (Men).csv,0,200,COMPLETED,"[TRAINING, PREDICTION]"


In [94]:
#Filter because storage is limited to 20GB
next_batch = catalog_metadata[0:3] #loc[[0,1,5]]
next_batch["datasetSize(MB)"].sum()

2

## Codespaceのストレージが次回ダウンロード分に対して十分な空き容量かの確認

In [95]:
raw = shutil.disk_usage(".")
free_mb = int(raw.free / (1_000_000))
print("空き容量: ", free_mb , "MB")
print("次回実行分に必要な空き容量確保できる：：", next_batch["datasetSize(MB)"].sum() < free_mb)


空き容量:  20798 MB
次回実行分に必要な空き容量確保できる：： True


## データセットをCodespaceのストレージにダウンロード

In [96]:
downloaded_datasets = download_datasets(next_batch, DIR, 4)
#downloaded_datasets

All downloads completed


## データセットをCodespaceのストレージにダウンロード

In [97]:
new_datasets = []
for cat in downloaded_datasets:
    record = {"name": cat["name"]}
    if cat["complete"]:
        resp = upload_dataset(cat, DIR=DIR)
        if "catalogId" in resp:
            record = record | resp
        new_datasets.append(record)

In [98]:
new_datasets

[{'name': '週次売上予測データセット_ヒストリカル.xlsx',
  'statusId': '878346e1-fbfc-4596-bab8-623a1408e462',
  'catalogId': '66975f719830d9199d3ce9d9',
  'catalogVersionId': '66975f719830d9199d3ce9da'}]

## 新しくアップロードしたデータセットのCatalogIDをプロジェクトデータに反映

In [108]:
next_batch_datasets = [""]

['週次売上予測データセット_ヒストリカル.xlsx']

In [99]:
next_batch_datasets = [d[1]["name"] for d in next_batch.iterrows()]
next_batch_projects = [p for p in projects if p["fileName"] in next_batch_datasets]
for idx, p in enumerate(next_batch_projects):
    matched_file = [ds for ds in new_datasets if ds["name"] == p["fileName"]][0]
    print("Matched file: ", matched_file)
    next_batch_projects[idx]["catalogId"] = matched_file["catalogId"]

Matched file:  {'name': '週次売上予測データセット_ヒストリカル.xlsx', 'statusId': '878346e1-fbfc-4596-bab8-623a1408e462', 'catalogId': '66975f719830d9199d3ce9d9', 'catalogVersionId': '66975f719830d9199d3ce9da'}


In [106]:
print("あデータセット登録は時間がかかる場合があります。ファイルサイズによっては数秒〜数分待ってかれプロジェクトのマイグレを始めてください。")

あデータセット登録は時間がかかる場合があります。ファイルサイズによっては数秒〜数分待ってかれプロジェクトのマイグレを始めてください。


In [107]:
#新規環境に登録したデータセットのメタデータを取得
new_catalog = dataset_metadata(next_batch_projects,env="TARGET")
#データセット登録完了しているかを確認
new_catalog[["name","processingState"]]

,name,processingState
0,週次売上予測データセット_ヒストリカル.xlsx,ERROR


## Project / Autopilot 実行開始

In [84]:
#next_batch_projects[0]

In [85]:
#AutopilotModeにインデックス１はない。
#https://docs.datarobot.com/en/docs/api/reference/public-api/projects.html
#mode_map = {0:"autopilot",2:"manual",3:"quick", 4:"comprehensive"}
adv_option_filter = ["downsampledMinorityRows", "downsampledMajorityRows","responseCap"]
for proj in next_batch_projects:
    print("プロジェクト開始： ", proj["projectName"])
    dr.Context.use_case = None
    new_proj = dr.Project.create_from_dataset(
        dataset_id=proj["catalogId"],
        project_name=proj["projectName"]	
        
    )
    #TypeError: AdvancedOptions.__init__()のエラーがスローされたら該当属性を以下の配列に追加する
    adv_options = {k:v for (k,v) in proj["advancedOptions"].items() if k not in adv_option_filter}
    adv_options = _convert_dictkeys_to_snake(adv_options)
    advanced_options = dr.AdvancedOptions(**adv_options)
    #partition_filter = ["datetimePartitionColumn", "useTimeSeries", "datetimeCol"]
    #partition = _convert_to_snake({k:v for (k,v) in proj["partition"].items() if k not in partition_filter})
    #partition_specs = None
    #if partition["cv_method"] == "datetime":
    #    print("This is a dateteime; instantiate object!")
    #    partition_specs = dr.DatetimePartitioningSpecification(**proj["partition"])
    #new_proj = new_proj.set_partitioning_method(**partition)
    
    try:
        new_proj = new_proj.analyze_and_model(
                    target=  proj["target"],
                    mode=    proj["autopilotMode"],
                
                    target_type=  proj["targetType"],
                    unsupervised_mode= proj["unsupervisedMode"],
                    unsupervised_type= proj["unsupervisedType"],
                    metric = proj["metric"],
                    positive_class = proj["positiveClass"],
                    advanced_options=advanced_options,
                    #partitioning_method=part_specs
        #未確認アイテム        
        #holdout_unlocked = proj["holdoutUnlocked"],

        )
    except dr.errors.ClientError as e:
        print("エラー発生：, ", e)

プロジェクト開始：  skin_lesion.zip
プロジェクト開始：  cal_housing.geojson
プロジェクト開始：  与信リスクのデータ_train.csv
プロジェクト開始：  商品の売上_学習.csv
プロジェクト開始：  skin_lesion.zip
プロジェクト開始：  MarketingTrain (Men).csv


In [52]:
##っ全てのプロジェクトが完了したら、プロジェクト・データセットをエキスポート
new_projects = export_projects(env="TARGET")
new_datasets_final = dataset_metadata(new_projects, env="TARGET")

In [53]:
new_datasets_final.to_csv(f"{DIR}/migrated-datasets.csv", index=False)

In [54]:
with open(f"{DIR}/migrated-projects.json", "w") as f:
    json.dump(new_projects, f, ensure_ascii=False, indent=4)

## 次回のバッチがある場合は移行済ファイルをStorageから削除してください